In [1]:
from symspellpy.symspellpy import SymSpell, Verbosity
import multiprocessing as mp
from time import time
import pandas as pd
import numpy as np
import random
import pickle
import spacy
import os
import re

from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.naive_bayes import MultinomialNB

# ML Model Building

In [2]:
df_train = pd.read_csv('train_sets/TF-IDF_MultinomialNB_train.csv',index_col='tweetid',encoding='utf-8-sig')
df_test = pd.read_csv('test_sets/TF-IDF_MultinomialNB_test.csv',index_col='tweetid',encoding='utf-8-sig')

In [3]:
#MultinomialNB

X_train,y_train = df_train['lemma_content_no'],df_train['sentiments']
X_test,y_test = df_test['lemma_content_no'],df_test['value']

my_pipe = Pipeline([
    ('vect', CountVectorizer(max_df=0.5,min_df=7,max_features=12000,ngram_range=(1,2))),
    ('tfidf', TfidfTransformer(norm='l2',use_idf=False)),
#     ('reduce',SelectKBest(chi2,k=4000)),
    ('clf', MultinomialNB(),),
])

#ajustamos el modelo at corpus de TASS
my_pipe.fit(X_train, y_train)

predictions = my_pipe.predict(X_test) 

print(confusion_matrix(y_test,predictions))  
print(classification_report(y_test,predictions, digits = 3))  
print(f"MultinomialNB Accuracy: {accuracy_score(y_test, predictions)}\n") 

[[369  66]
 [ 90 253]]
              precision    recall  f1-score   support

           N      0.804     0.848     0.826       435
           P      0.793     0.738     0.764       343

   micro avg      0.799     0.799     0.799       778
   macro avg      0.799     0.793     0.795       778
weighted avg      0.799     0.799     0.799       778

MultinomialNB Accuracy: 0.7994858611825193



In [4]:
# Saving model to disk
# pickle.dump(my_pipe, open('models/TF-IDF_MNB.pkl','wb'))

# Web Service

In [5]:
def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r'\bq\b|\bk\b', 'que', text) # replace q or x with que
    text = re.sub(r'\bd\b', 'de', text) # replace d with de
    text = re.sub(r'\bx\b', 'por', text) # replace x with por
    text = re.sub(r'\btmb\b', 'también', text) # replace tmb with tambien
    text = re.sub(r'\bbb\b', 'bebé', text) # replace bb with bebe
    
    duplicates = re.compile(r'([^(c,l,n,r)L0])\1{1,}')
    double_clnr = re.compile(r"(.)\1{2,}")
    while duplicates.search(text)!=None:
        text = text.replace(duplicates.search(text).group(),duplicates.search(text).group()[0]) #remove multiple letters
    text = double_clnr.sub(r"\1\1", text) #except double c, l, n, and r
       
    text = re.sub(r'([ja]{5,}|[je]{5,}|[ji]{5,}|[ha]{5,}|[he]{5,})', 'jaja', text)  # remove dirty laughs

    text = re.sub(r'(\.|,|:|;|!|\?|\[|\]|\(|\))[A-Za-z0-9]+', ' ', text)  # replace simbols between words with spaces
    text = re.sub(r'\d+', '', text) #remove numbers
    
    text = re.sub(r'[%s]' % re.escape("""¿¡!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~…"""), '', text)  # remove punctuations
    text = re.sub(r'\b[^aeyou]\b', ' ', text) # remove single char
    text = re.sub('\s+', ' ', text)  # remove extra whitespace
    
    
    text = text.encode('latin', 'ignore').decode('latin')
    
    max_edit_distance_lookup = 3
    text = sym_spell.lookup_compound(text,max_edit_distance_lookup)[0].term
    
    
    tokens= nlp(u""+text)
    new_text= ' '.join([t.lemma_ for t in tokens])
    
    
    return [new_text]

In [6]:
nlp = spacy.load("es_core_news_md")

with open('resources/stopwords-es.txt','r',encoding='utf-8-sig') as f:
    my_stop_words = f.read().splitlines()
    
with open('resources/spanish_words.txt','r',encoding='utf-8-sig') as f:
    spanish_words = f.read().splitlines()
    
print(len(my_stop_words))
    
no_remove= "bastante,bien,buen,buena,buenas,bueno,buenos,casi,\
demasiado,debe,deben,debido,ex,hubiera,hubierais,hubieran,hubieras,\
hubieron,hubiese,hubieseis,hubiesen,hubieses,hubimos,hubiste,\
hubisteis,hubiéramos,hubiésemos,hubo,mal,más,mucha,muchas,mucho,\
muchos,muy,nada,nadie,ni,ninguna,ningunas,ninguno,ningunos,ningún,\
no,nueva,nuevas,nuevo,nuevos,nunca,peor,pero,pesar,poca,pocas,\
poco,pocos,podeis,podemos,poder,podria,podriais,podriamos,podrian,\
podrias,podrá,podrán,podría,podrían,quiza,quizas,quizá,quizás,\
siempre,sola,solamente,solas,solo,solos,tendremos,tendrá,tendrán,\
tendrás,tendré,tendréis,tendría,tendríais,tendríamos,tendrían,\
tendrías,tened,teneis,tenemos,tener,tenga,tengamos,tengan,tengas,\
tengo,tengáis,tenida,tenidas,tenido,tenidos,teniendo,tenéis,\
tenía,teníais,teníamos,tenían,tenías,tiempo,tiene,tienen,tienes,\
tuve,tuviera,tuvierais,tuvieran,tuvieras,tuvieron,tuviese,\
tuvieseis,tuviesen,tuvieses,tuvimos,tuviste,tuvisteis,tuviéramos,\
tuviésemos,tuvo,si,decir,dice,dicen,dicho,dijeron,dijo,buen,pesar".split(',')

print(len(no_remove))

my_stop_words=[s for s in my_stop_words if s not in no_remove]
my_stop_words = frozenset(my_stop_words)

print(len(my_stop_words))

739
139
602


In [7]:
sym_spell = SymSpell(
    max_dictionary_edit_distance=3,
    prefix_length=7,
    count_threshold=1,
    compact_level=5,
)

sym_spell.load_dictionary(corpus='resources/es_real_freq_full.txt',term_index=0,count_index=1,encoding='utf-8')

True

In [8]:
text_examples = ["nada, simplemente desepcionado de como.se.dan las cosas",
"las personas a quienes atiendo me hacen enojar porque no entienden mis instrucciones y hacen lo que quieren",
"claro! mi problemática radica en que se me dificulta entablar relación con mis compañeros de trabajo. Motivo por el cual el ambiente laboral es tenso",
"me molesta la falta de compromiso de un miembro de mi familia para ayudar en casa",
"me quede sin trabajo hace días",
"me hago ilusiones con personas muy rapido",
"que ya no pude terminar mis estudios superiores y no he podido encontrar  jn mejor empleo",
"voy a comezar a vivir con mi esposo, no tenemos mucha estabilidad economica, es injusto trabajar tanto y aun asi tener problemas economicos",
"la pareja que tenia me mintio y engaño mucho",
"ayer fue cumpleaños de mi novio y lo mande felicitar por fb y borro la felicitacion y mi foto y eso me molesto mucho",
"hoy me robaron dinero de un tramite de la oficina eso me causo preocupacion y tuve mucho trabajo, me acelere mucho y con la tension del problema... me causo estres y al final de la jornada cansancio y un poco de ansiedad",
"No me esfuerzo lo suficiente",
"fracaso de pareja",
"no puedo terminar los trabajos a  tiempo",
"fallecio una persona muy querida, era como si fuera parte de mi familia ... esto me hace sentir triste",
"almejo se va a otros lados en lugar de estar en el proyecto del eneit",
"me siento tonta",
"un posible trabajo me poner en la necesidad de cambiar de residencia",
"la verdad ni yo se que es lo que pasa,.de repente esroy.bien y de rato unos 'bajones' feos",
"siento que voy a reprobar",
"que no me atiende",
"me siento insegura al abordar una conversación me da miedo hablar algunas cosas",
"me molesta mucho que mi esposo se tome las cosas con tanta calma... que sea tan decidioso... que tenga que estarlo presionando para que haga las cosas",
"mi hijo se va a regresar con su padre porque quiere estudiar allá, yo me siento impotente enojada porque no logre los planes de cuando llegamos aquí y ahora el se va se que es para superarse pero creo que no di el 100 con él",
"Mi novia no me ayuda con las labores del hogar",
"no es justo que yo trabaje esforzandome mucho para terminar con mas rapidez... y las compañeras una sea muy lenta y la otra sea tan conchuda eso mr irrita bastante que nos le llamen la atencion me da mucho coraje",
"estoy como aburrida..solo quiero irme a acostar..nada me llama la atencion",
"mesiento inseguro esto mebuelbe distraido esto metrae prpblemas en mi trabajo",
"planeo realizarla por la tarde del día",
"nada simplemente aceptar que tengo problemas psicológicos",
"No tuve tiempo de hacerla",
"no me di el tiempo para hacerlo",
"pienso realizarla un rato mas",
"falta de tiempo",
"no me organizo con mis tiempos, me gana el cansancio o la flojera",
"cambie el dia porque tube hoy cosas q hacer..espero que mañana se pueda",
"Aun no termina el día",
"si la realice solo que algunas veces no se como darle la respuesta.la actividad es con mi hijo ayer le dije que lo quiero mucho y que es lo mas valioso que tengo, hoy le dije que termine sus materias que yo siempre lo voy apoyar",
"sali a convivir con mis hijos",
"No me puse de acuerdo con la amiga para salir",
"estaba lloviendo y tuve que regresar mas rapido",
"no necesite ayuda",
"La lluvia",
"Falta de tiempo",
"Cambio de planes",
"No hubo obstáculo",
"pues ver lo positivo de ello",
"considere mas opciones",
"no lo sé",
"que busque empleo donde se sienta mejor",
"que es muy valioso que puede salir adelante solo",
"que le echara ganas que nadie nace enseñado",
"no te preocupes todo va a mejorar, asi es como comienzan las parejas pero es parte del matrimonio, no te desanimes echenle  ganas",
"que buscara otra y se olvidara de ella",
"que vale mucho,,,pero no ha llegado la persona correcta",
"si tu pareja te ignora es porque no le importas",
"abraza tu ansiedad no pasa nada, estas en tx todo va fluyendo poco a poco respira... piensa en detectar que situacion te pone asi... disfruta cada momento relajate",
"No te excuses",
"que no sea como yo",
"etres, presion",
"no tengo explicación",
"que tuviera comunicacion",
"que el no ya es seguro hay que buscar el si",
"ten paciencia poco a poco... todo saldra bien",
"que se vea en mi y no cometa los mismos errores pues no se beneficia en nada y se perjudica a los hijos",
"Llegar a un acuerdo para repartir las labores",
"nose",
"no se..no tengo ganas de dar ninguna explicación",
"necesito integrarme mas a las actividades en grupo y quitar las barreras que implemento para que no me conozcan",
"porque no puedo desempeñarme en cualquier área de trabajo",
"que dificil es comenzar una vida juntos, muchos obstaculos",
"tengo miedo de tener crisis de ansiedad",
"miedo a sentir ansiedad",
"que no podremos salir adelante con tantos gastos... yo no puedo sola",
"no he luchado lo suficiente para darles a mis hijos lo que necesitan y la atención que merecen",
"Es necesario que me ayude con algunas labores",
"darme un tiempo para refelxionar y no nada mas irme a la primera imprecion o pensamiento.",
"no centrarme solo en una solución, sino ver mas opciones y pensar de manera diferente",
"que el preocuparme por las acciones de los otros altera mi comportamiento. necesito centrarme en mí y mis actividades para que mi bienestar no se vea afectado.",
"No tomar importancia de las acciones de los demás",
"no pensar negativamente",
"a valorarme mas y aceptar las cosas",
"que debo aceptarme tal y cual soy",
"mmmm no tener un pensamiento o vision tan cuadrada... aprender a observar y tomar calma ante todo",
"que no debo sufrir por acciones ajenas",
"que no estoy sola....y que valgo  mucho",
"pues es verdad una persona no es todo el mundo hay personas q de verdad nos aprecian..no podemos cambiar la forma de ser de nadie..solo la nuestra",
"No pienses,actua",
"que valgo por mi pefsona",
"alrender a escuchar y respetar las opiniones de los demas",
"las cosas en que fallo no determinan quien soy",
"que los pensamientos me hacen sentir mal",
"que me debo ocupar mas de mi",
"sigo molesta",
"que no debo permitir que los comentarios de mis hijos me lastimen ya que para mi son mi prioridad",
"Llegar a un acuerdo",
"seguir adelante",
"respirar pensar con mas calma y mas friamente sobre el asunto para poder decidir que hacer",
"si no me gusta la vida q estoy viviendo que cambie un poco que la mejore"]

In [9]:
# my_text= "No tiene sentido seguir viviendo. Nadie me quiere."

my_text = "".join(random.sample(text_examples,1))

start1 = time()
print(f"Dirty_text: {my_text}")
proba = my_pipe.predict_proba([my_text])[0][1]
print(f"Probability: {proba}")
value = ("P+" if proba>=0.8 else "P" if 0.66<=proba<0.8
         else "NEU" if 0.56<=proba<0.66 else "N" if 0.26<=proba<0.56 else "N+")
print(f"Value: {value}")
print(f"Process took {time()-start1} seconds to finish\n")

start2 = time()
print(f"Clean_text: {clean_text(my_text)[0]}")
proba2 = my_pipe.predict_proba(clean_text(my_text))[0][1]
print(f"Probability: {proba2}")
value = ("P+" if proba2>=0.8 else "P" if 0.66<=proba2<0.8
         else "NEU" if 0.56<=proba2<0.66 else "N" if 0.26<=proba2<0.56 else "N+")
print(f"Value: {value}")
print(f"Process took {time()-start2} seconds to finish")

Dirty_text: las cosas en que fallo no determinan quien soy
Probability: 0.3915786158725784
Value: N
Process took 0.12236952781677246 seconds to finish

Clean_text: los coser en que fallir no determinar quien ser
Probability: 0.4210839251467557
Value: N
Process took 0.41282153129577637 seconds to finish
